In [ ]:
%matplotlib inline
from __future__ import print_function

try:
    xrange
except NameError:
    xrange = range

import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.decomposition import LatentDirichletAllocation
import lda
import scipy.sparse as sparse
from collections import Counter

<h2>LDAを使ってみよう</h2>

LDAとはトピックモデルの手法の一つで、Pythonでは以下のようなライブラリでLDAを使用することができます。
* scikit-learn (0.17以降）
* lda
* gensim

ちなみに、LDAというと、Linear Discriminant Analysis(線形判別分析）と思う人もいるので、注意（実際、scikit-learnでLDAは線形判別分析と表しており、トピックモデルのLDAはLatentDirichletAllocationという名前）

今回はlda_datasetフォルダにある2つのJSONファイルを使用します。これらはカメリオで収集した2016/10/21の記事をランダムに1000件サンプリングし、形態素解析した結果ですが、少しだけ異なる処理で2種類作成しました。document_word_data.jsonは名詞を全て含んだもの、document_word_data_pnoun.jsonはproper noun、つまり、固有名詞のみのデータです。これら2つを比較するのも面白いです。


<h2>LDAのインプットデータを作ろう</h2>

ここでのゴールは、文書ID x 単語の行列（成分は出現頻度）を作ることです。ここで注意しなければならないのは、この行列は凄まじくスパースである、つまり、ほとんどの成分が0であるということです。0であるということをデータ上に保存するのはもったいないので、0以外の成分のみをデータとして持っておきたいときに役立つのがスパース行列です。

いろいろな表現方法がありますが、ここでは、List of List (LIL）という方法でデータを保存します。
LILはとても簡単です。

例えば、以下のようなAという行列があるとき、

\begin{equation*}
A = \left[
    \begin{array}{rrr}
      0 & 2 & 0 & 0 \\
      1 & 0 & 0 & 1 \\
      0 & 0 & 0 & 1
    \end{array}
  \right]
\end{equation*}

LILは、ゼロではない成分の行と列、その中の値を保存することで、上記の行列の情報を保存します。
なので、上の$A$をLIL形式で表現すると、
\begin{equation*}
(1, 2) = 2 ~~ (1行目2列目の成分が2)\\
(2,1 ) = 1 ~~ (2行目1列目の成分が1)\\
(2,4 ) = 1 ~~ (2行目4列目の成分が1)\\
(3,4 ) = 1 ~~ (3行目4列目の成分が1)\\
\end{equation*}
となります。

では、LDAのインプットデータを作る場合も同様に考えられて、行に文書のインデックス、縦に単語のインデックスをとって、値があるところだけをLIL形式でデータを保存すれば良いということになります。

さて、そこまでやってみましょう。まずデータを読み込みます。

In [ ]:
with open("./dataset/document_word_data.json", "r") as f:
    doc_data= json.load(f)

all_doc_index = doc_data.keys()
print("Total Documents: ", len(all_doc_index))

この読み込んだjsonデータはこんな感じです。

In [ ]:
", ".join(doc_data['715'])

単語のインデックスを作るために、全ての単語のリストを作ります。

In [ ]:
all_vocab = []
for doc_idx in all_doc_index:
    all_vocab += doc_data[doc_idx]

#重複を消すためにsetしてlistにする
all_vocab = list(set(all_vocab))
vocab_num = len(all_vocab)
print("Vocablary Number: ", vocab_num)

LIL形式の行列を定義します。ScipyにはLIL形式でデータを保存するための機能がありますので、そちらを使います（ScipyはLIL以外のスパース行列のフォーマットをサポートしていますが）。

ここからは学習データと学習後に適用するデータ（ここでは便宜的にテストデータ）に分けましょう。
LDAは教師あり機械学習ではないので、ここでのテストデータの役割は学習済みのモデルに当てはめるデモ用という位置付けです。

In [ ]:
all_doc_index_ar = np.array(list(all_doc_index))

train_portion = 0.7
train_num = int(len(all_doc_index_ar) * train_portion)

np.random.shuffle(all_doc_index_ar)
train_doc_index = all_doc_index_ar[:train_num]
test_dox_index = all_doc_index_ar[train_num:]

先にからっぽのスパース行列を定義します。

In [ ]:
train_A = sparse.lil_matrix((len(train_doc_index), len(all_vocab)), dtype = np.int)
test_A = sparse.lil_matrix((len(test_dox_index), len(all_vocab)), dtype = np.int)

ListからNumpyのArrayに直します。

In [ ]:
all_vocab_ar = np.array(all_vocab)
train_doc_index_ar = np.array(train_doc_index)
test_doc_index_ar = np.array(test_dox_index)

スパース行列に成分を入れていきます。

In [ ]:
#学習用
train_total_elements_num = 0
for i in xrange(len(train_doc_index)):
    doc_idx = train_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        train_A[i, word_idx] = row_data[word]
        train_total_elements_num += 1
print("Train total elements num :", train_total_elements_num)


#テスト用
test_total_elements_num = 0
for i in xrange(len(test_dox_index)):
    doc_idx = test_dox_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        test_A[i, word_idx] = row_data[word]
        test_total_elements_num += 1
print("Test total elements num :", test_total_elements_num)

## 実際にLDAを適用してみよう (Scikit-learnを使った例）

ここではsikit-learnの例を示します。scikit-learnのLDAはオンライン変分ベイズ法という推定方法を用いています。オンラインと名前が付いているので勘が良い方ならお気づきだと思いますが、SGDと同じように部分的にフィット(partial fit)させて、捨てるという形での推定が可能、つまり追加で学習がしやすいというところが特徴です。

In [ ]:
model1 = LatentDirichletAllocation(n_topics=20, 
                                doc_topic_prior= 0.001,
                               topic_word_prior=0.5,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [ ]:
model1.fit(train_A)

まずトピック x 単語を見てみましょう

In [ ]:
normalize_components = model1.components_ /model1.components_.sum(axis=0)

In [ ]:
#http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html　より
n_top_words = 20
for topic_idx, topic in enumerate(normalize_components):
    print("Topic #%d:" % topic_idx)
    print(" ".join([all_vocab_ar[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

文書 x トピック行列側も見てみましょう。

In [ ]:
doc_topic_data = model1.transform(train_A)
doc_topic_data

scikit-learnのLDAはどうやら正規化されていないため、正規化した上で、1つ目の文書がどのトピックから来ている単語が多いのかを見てみましょう。

In [ ]:
normalize_doc_topic_data = doc_topic_data/doc_topic_data.sum(axis=1, keepdims=True)

In [ ]:
for topic_idx, prob in enumerate(normalize_doc_topic_data[0]):
    print("Topic #%d: probality: %f" % (topic_idx, prob))

当てはまりの度合いを測る指標の1つとして対数尤度(Loglikelihood)があります。できるだけ0に近いほどよく当てはまっていることになりますが、「X以上あれば精度が良い」と言えるような絶対的な指標ではなく、ハイパーパラメーターを変えた時などの相対的な比較に使うものだという点を気をつけてください。

In [ ]:
loglikelihood = model1.score(test_A)
ppl = model1.perplexity(test_A)
print("対数尤度: ", loglikelihood)
print("Perplexity: ", ppl)

テストデータに当てはめてみましょう。

In [ ]:
test_doc_topic_data = model1.transform(test_A)
normalize_test_doc_topic_data = test_doc_topic_data/test_doc_topic_data.sum(axis=1, keepdims=True)
for topic_idx, prob in enumerate(normalize_test_doc_topic_data[0]):
    print("Topic #%d: probality: %f" % (topic_idx, prob))

<h2> LDAを適用してみよう (ldaパッケージを使った場合)</h2> 

ldaパッケージはより高速なGibbs samplingという手法を使って推定するパッケージです。ただし、これはオンライン学習はできない、つまりバッチ処理しかできません。また、変分ベイズ法による推定とGibbs samplingのどちらが精度が良いかというのはわかりません。

In [ ]:
model2 = lda.LDA(n_topics=20, n_iter=1500, random_state=1, alpha=0.5, eta=0.5)

In [ ]:
model2.fit(train_A)

In [ ]:
topic_word = model2.topic_word_
n_top_words = 20
for topic_idx, topic in enumerate(topic_word):
    print("Topic #%d:" % topic_idx)
    print(" ".join([all_vocab_ar[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

今回も精度として対数尤度を見てみましょう

In [ ]:
model2.loglikelihood()

文書 x トピック行列を見てみましょう。

In [ ]:
doc_topic_data2 = model2.transform(train_A)
for topic_idx, prob in enumerate(doc_topic_data2[0]):
    print("Topic #%d: probality: %f" % (topic_idx, prob))

<h2>（参考）ディリクレ分布の挙動</h2>

In [ ]:
alpha = 10.0
K = 6
sampled_probs = np.random.dirichlet([alpha for i in range(K)])
for i, prob in enumerate(sampled_probs):
    print("サイコロ %d面  確率: %.2f"%(i+1, prob))

<h2>（参考）潜在ディリクレ分配法　少しばかり数式を使った解説</h2>
<p>
ある1つの文書データdの生成過程を数式で書くと,<br><br>
文書データdの単語の個数を$N_d$とします。
<br>
<br>
<h4>1. トピックの箱の選ばれやすさを決める</h4>
トピックの確率分布はディリクレ分布に従うものとします。つまり、$\theta_d \sim Dir(\alpha)$ です。この式の意味は、どのトピックの箱がどの程度選ばれるかが$\theta_d$でハイパーパラメーターが$\alpha$ということになり、$\alpha$が1の時、どのトピックの箱が選ばれやすいかは完全にランダムになります。
<br>
<br>
次に各単語について、以下の手順を想定します。
<h4>2. トピックの箱を決める</h4>
トピックの箱を決めるのにカテゴリ分布（多項分布）を想定します（ディリクレ分布とカテゴリ分布は共役だったことを思い出してください）。つまり、$z_{dn} \sim Category(\theta_d)$　を仮定します（$z_{dn}$はトピックの箱の番号だと思ってください）

<h4>3. 単語カードを引く</h4>
トピックの箱を決めた時に、そのトピックの箱から単語カード$w_n$を引く確率を$P(w_{dn} | \phi_{z_n})$ ($n = 1,2, 3, \cdots,  N_d$)と書きます。そして、この$P(w_dn | \phi_{z_{dn}})$を多項分布、$\phi_{z_dn}$をディリクレ分布で表現します。つまり、
\begin{equation*}
\phi_{z_{dn}} \sim   Dir(\beta)
\\
w_{dn} \sim Category(\phi_{z_{dn}} )
\end{equation*}
となります。
<br>
<br>
上記を組みわせて文書データdの生成仮定を以下のように書くことができます。

\begin{equation*}
P(d | \alpha, \bf \beta ) = \int P(\theta_d | \alpha) \prod_{n=1}^{N_d}P(z_{dn} |\theta_d) P(w_{dn} | \phi_{z_{dn}}) P(\phi_{z_{dn}} | \beta)d\theta_d
\end{equation*}
そして、M個の文書データ全体を$D$とすると以下のようになります。
\begin{equation*}
P(D | \alpha, \bf \beta ) = \prod_{d=1}^{M}\int P(\theta_d | \alpha) \prod_{n=1}^{N_d}P(z_{dn} |\theta_d) P(w_{dn} | \phi_{z_{dn}}) P(\phi_{z_{dn}} |
\end{equation*}

若干、デフォルメしてありますが、上記を模したものが以下です。
まず、以下のようにサッカー、音楽、経済の3つのトピックがあるものと仮定します。そして、それぞれのトピックに対し、どの単語がどの程度でやすいのかをディリクレ分布よりサンプリングします。その結果を「phi_トピック名」という配列に入れます。

In [ ]:
topic_soccer = ["サッカー", "本田圭佑", "セリエA", "日本代表", "香川真司"]
topic_music = ["音楽", "ライブ", "ギター", "コンサート", "最高", "武道館"]
topic_econ = ["経済", "株価", "日本企業", "スタートアップ", "Fintech", "マーケティング"]
topics = [topic_soccer, topic_music, topic_econ]

beta = 0.5
phi_soccer = np.random.dirichlet([beta for i in range(len(topic_soccer))])
phi_music = np.random.dirichlet([beta for i in range(len(topic_music))])
phi_econ = np.random.dirichlet([beta for i in range(len(topic_econ))])

phi = [phi_soccer, phi_music, phi_econ]

print("サッカートピックのPhi: ", phi_soccer)
print("音楽トピックのPhi: ", phi_music)
print("経済トピックのPhi: ", phi_econ)

In [ ]:
N = 10  #単語数
alpha = 0.5 #トピック分布のハイパーパラメータ
K = 3 #トピック数
theta = np.random.dirichlet([alpha for i in range(3)])
print("Theta :", theta)

generate_doc = []

for i in range(N):
    selected_topic_ar = np.random.multinomial(1.0, theta)
    selected_topic_idx = np.where(selected_topic_ar==1)[0][0]
    
    
    selected_word_ar = np.random.multinomial(1, phi[selected_topic_idx])
    selected_word_idx = np.where(selected_word_ar==1)[0][0]
    selected_word = topics[selected_topic_idx][selected_word_idx]
    generate_doc.append(selected_word)
    print("i = ", i, "Selected topic:", selected_topic_idx, "Selected word:", selected_word)

    
print('Finally, generated document is "', " ".join(generate_doc))

このような生成プロセスを経て、我々が目にしている文書が得られていると仮定するわけです。<br>
さて、ではLDAで推定したいものは何でしょうか？それは、各トピックと$\theta$と$\phi$とかなのですが、その推定方法は、主に2つあります。一つは変分ベイズ
法による推定、もう一つはGibbs samplingによる推定です。興味がある方は調べてみてください。